<a href="https://colab.research.google.com/github/mihirkumar28/Deep-Generative-Learning/blob/main/Text_Generation_using_LSTM(Simple).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np 
path = keras.utils.get_file('nietzsche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print(len(text))

606208/600901 [==============================] - 0s 0us/step
600893


In [2]:
maxlen=60
step=3
sentences=[]
next_chars=[]
for i in range(0,len(text)-maxlen,step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print(sentences[0])
print(next_chars[0])

preface


supposing that truth is a woman--what then? is the
r


In [3]:
unique_chars=sorted(list(set(text)))
char_indices = dict((unique_char,unique_chars.index(unique_char)) for unique_char in unique_chars)
x=np.zeros((len(text),maxlen,len(unique_chars)),dtype=np.bool)#Input_Vector
y=np.zeros((len(text),len(unique_chars)),dtype=np.bool)#Output_Vector
#Vectorization
for i,sequence in enumerate(sentences):
  for j,char in enumerate(sequence):
    x[i,j,char_indices[char]]=1
  y[i,char_indices[next_chars[i]]]=1

In [4]:
from keras import layers
model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(unique_chars))))
model.add(layers.Dense(len(unique_chars),activation='softmax'))

In [5]:
optimizer=keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

In [6]:
def sample(preds,entropy):
  preds=np.asarray(preds).astype('float64')
  preds=np.log(preds)/entropy
  exp_preds=np.exp(preds)
  preds=exp_preds/np.sum(exp_preds)
  probabilities=np.random.multinomial(1,preds,1)
  return np.argmax(probabilities)

In [ ]:
import random
import sys

for epoch in range(1,20):
  print('\nEpoch Number',epoch,"\n")
  model.fit(x,y,batch_size=128,epochs=1)
  start=random.randint(0,len(text)-maxlen-1)
  generated_text=text[start:start+maxlen]

  for entropy in [0.2,0.5,1.0,1.2]:
    print("\n\nStochasticity",entropy,"\n\nInput\n\n")
    sys.stdout.write(generated_text)
    #for i in range(100):
    sampled=np.zeros((1,maxlen,len(unique_chars)))
    for t,chars in enumerate(generated_text):
      sampled[0,t,char_indices[char]]=1
      
    preds=model.predict(sampled,verbose=0)[0]
    next_index=sample(preds,entropy)
    next_char=unique_chars[next_index]

    generated_text+=next_char
    generated_text=generated_text[1:]
    print("\n\nOutput\n\n")
    sys.stdout.write(generated_text)


Epoch Number 1 

4695/4695 [==============================] - 27s 6ms/step - loss: 1.0777


Stochasticity 0.2 

Input


have been termed a school of distrust, still
more of disdain

Output


ave been termed a school of distrust, still
more of disdain 

Stochasticity 0.5 

Input


ave been termed a school of distrust, still
more of disdain 

Output


ve been termed a school of distrust, still
more of disdain h

Stochasticity 1.0 

Input


ve been termed a school of distrust, still
more of disdain h

Output


e been termed a school of distrust, still
more of disdain ha

Stochasticity 1.2 

Input


e been termed a school of distrust, still
more of disdain ha

Output


 been termed a school of distrust, still
more of disdain ha 
Epoch Number 2 

4695/4695 [==============================] - 28s 6ms/step - loss: 1.0782


Stochasticity 0.2 

Input


and retiring, a noble weakling who
revelled in nothing but a

Output


nd retiring, a noble weakling who
revelled in nothing but as

Stochastici

In [ ]:
generated_text="new faculty, and the jubilation reached its climax when kant"
sampled=np.zeros((1,maxlen,len(unique_chars)))
for t,chars in enumerate(generated_text):
  sampled[0,t,char_indices[char]]=1
      
preds=model.predict(sampled,verbose=0)[0]
next_index=sample(preds,entropy)
next_char=unique_chars[next_index]

generated_text+=next_char
generated_text=generated_text[1:]
sys.stdout.write(generated_text)